# ASSIGNMENT

# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [199]:
# Hard to do this without ASLR disabled:
# echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
# /etc/sysctl.d/01-disable-aslr.conf

In [4]:
prog_name = "brainpan.exe"
ssh = False

print(prog_name)

brainpan.exe


In [9]:
!chmod 777 {prog_name}

In [10]:
import pexpect
import sys
import time

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)


ip_addr = '10.10.214.213'
command = 'r2 -d ' + prog_name
if ssh:
    rp = pexpect.spawnu('ssh user1@%s' % ip_addr, encoding='utf-8')
else:
    rp = pexpect.spawnu(command, encoding='utf-8')

@register_line_magic
def r(line):
    print(line)
    rp.sendline(line)
    time.sleep(.5)
    end = ""
    while not "]>" in end:
        read = rp.read_nonblocking(100000)
        print(read)
        end = read[-50:]

@register_line_magic
def ro(line):
    print(rp.read_nonblocking(100000))

@register_line_magic
def ri(line):
    rp.sendline(line)
    time.sleep(.5)
    print(rp.read_nonblocking(100000))

In [11]:
if ssh:
    %ri user1password

In [12]:
if ssh:
    %ri {command}

In [13]:
%r aa;e asm.syntax=att; afl;

aa;e asm.syntax=att; afl;
Process with PID 2760 started...
= attach 2760 2760
bin.baddr 0x55b7034bb000
Using 0x55b7034bb000
asm.bits 64
[Analyze all flags starting with sym. and entry0 (aa)
[x] Analyze all flags starting with sym. and entry0 (aa)

0x55b7034bf760    1 42           entry0
0x55b7034d8fc0    1 96           reloc.__libc_start_main
0x55b7034bf030    1 6            sym.imp.sigprocmask
0x55b7034bf040    1 6            sym.imp.raise
0x55b7034bf050    1 6            sym.imp.free
0x55b7034bf060    1 6            sym.imp.strcasecmp
0x55b7034bf070    1 6            sym.imp.abort
0x55b7034bf080    1 6            sym.imp.__errno_location
0x55b7034bf090    1 6            sym.imp._exit
0x55b7034bf0a0    1 6            sym.imp.strcpy
0x55b7034bf0b0    1 6            sym.imp.qsort
0x55b7034bf0c0    1 6            sym.imp.isatty
0x55b7034bf0d0    1 6            sym.imp.sigaction
0x55b7034bf0e0    1 6            sym.imp.strtod
0x55b7034bf0f0    1 6            sym.imp.strchrnul
0x55b7034bf1

In [15]:
%r pdf @main

pdf @main
            ; DATA XREF from entry0 @ 0x55b7034bf77d
            ;-- section..text:
┌ 464: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_114h @ rsp+0x4
│           ; var int64_t var_110h @ rsp+0x8
│           ; var int64_t var_fch @ rsp+0x1c
│           ; var int64_t var_f8h @ rsp+0x20
│           ; var int64_t var_d8h @ rsp+0x40
│           ; var int64_t var_10h @ rsp+0x108
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           0x55b7034bf590      53             pushq %rbx              ; [14] -r-x section size 71793 named .text
│           0x55b7034bf591      4881ec100100.  subq $0x110, %rsp
│           0x55b7034bf598      897c2404       movl %edi, var_114h     ; argc
│           0x55b7034bf59c      4889742408     movq %rsi, var_110h     ; argv
│           0x55b7034bf5a1      64488b042528.  movq %fs:0x28, %rax
│           0x55b7034bf5aa      488984240801.  movq %rax, var_10h
│           0x55b7034bf5b2      31c0        

In [23]:
%r pd @0x55b7034c82f0

pd @0x55b7034c82f0
            ; CALL XREF from main @ 0x55b7034bf715
            0x55b7034c82f0 b    4157           pushq %r15
            0x55b7034c82f2      4531ff         xorl %r15d, %r15d
            0x55b7034c82f5      4156           pushq %r14
            0x55b7034c82f7      4189fe         movl %edi, %r14d
            0x55b7034c82fa      4155           pushq %r13
            0x55b7034c82fc      4189fd         movl %edi, %r13d
            0x55b7034c82ff      4183e601       andl $1, %r14d
            0x55b7034c8303      4154           pushq %r12
            0x55b7034c8305      4183f501       xorl $1, %r13d
            0x55b7034c8309      4c8d25203a01.  leaq 0x55b7034dbd30, %r12
            0x55b7034c8310      55             pushq %rbp
            0x55b7034c8311      31ed           xorl %ebp, %ebp
            0x55b7034c8313      53             pushq %rbx
            0x55b7034c8314      4883ec38       subq $0x38, %rsp
            0x55b7034c8318      64488b042528.  movq %fs:0x28, %ra

In [18]:
# %r pdf @0x55b7034d0710

pdf @0x55b7034d0710
p: Cannot find function at 0x55b7034d0710d0710
[0x7f1d6d0d7090]> 


# R2: Set Break Points:


In [333]:
# %r db 0x00400527
# %r db 0x00400552

db 0x00400527
[0x7ffff7dd8ef0]> K
db 0x00400552
[0x7ffff7dd8ef0]> K


In [25]:
%r dc

dc


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 184: invalid start byte

In [335]:
%r pdf

pdf
            ;-- rip:K
┌ 61: sym.copy_arg (int64_t arg1);
│           ; var int64_t var_98h @ rbp-0x98
│           ; var int64_t var_90h @ rbp-0x90
│           ; arg int64_t arg1 @ rdi
│           ; CALL XREF from main @ 0x40058b
│           0x00400527 b    55             pushq %rbp
│           0x00400528      4889e5         movq %rsp, %rbp
│           0x0040052b      4881eca00000.  subq $0xa0, %rsp
│           0x00400532      4889bd68ffff.  movq %rdi, var_98h          ; arg1
│           0x00400539      488b9568ffff.  movq var_98h, %rdx
│           0x00400540      488d8570ffff.  leaq var_90h, %rax
│           0x00400547      4889d6         movq %rdx, %rsi
│           0x0040054a      4889c7         movq %rax, %rdi
│           0x0040054d      e8defeffff     callq sym.imp.strcpy        ; char *strcpy(char *dest, const char *src)
│           0x00400552 b    488d8570ffff.  leaq var_90h, %rax
│           0x00400559      4889c7         movq %rax, %rdi
│           0x0040055c      e8dffeffff

In [336]:
# %r ds

# Inspect Variables:

In [337]:
%r px @ rsp

px @ rsp
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7fffffffe408  9005 4000 0000 0000 08e5 ffff ff7f 0000  ..@.............
0x7fffffffe418  0000 0000 0200 0000 a005 4000 0000 0000  ..........@.....
0x7fffffffe428  2a30 a4f7 ff7f 0000 0000 0000 0000 0000  *0..............
0x7fffffffe438  08e5 ffff ff7f 0000 0000 0400 0200 0000  ................
0x7fffffffe448  6405 4000 0000 0000 0000 0000 0000 0000  d.@.............
0x7fffffffe458  1c7a 28cb 7493 f62b 5004 4000 0000 0000  .z(.t..+P.@.....
0x7fffffffe468  00e5 ffff ff7f 0000 0000 0000 0000 0000  ................
0x7fffffffe478  0000 0000 0000 0000 1c7a 0808 0b6c 09d4  .........z...l..
0x7fffffffe488  1c7a ac9f bc7c 09d4 0000 0000 0000 0000  .z...|..........
0x7fffffffe498  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7fffffffe4a8  20e5 ffff ff7f 0000 30e1 fff7 ff7f 0000   .......0.......
0x7fffffffe4b8  5676 def7 ff7f 0000 0000 0000 0000 0000  Vv..............
0x7fffffffe4c8  0000 0000 000

In [338]:
%r dc

dc
hit breakpoint at: 400552
[0x00400552]> 


In [339]:
# we need to hit 0x7ffc03344dc8 + 4
# safe jump: f00411468
%r px @ rbp-0x98

px @ rbp-0x98
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7fffffffe368  5ae7 ffff ff7f 0000 4141 4141 4141 4141  Z.......AAAAAAAA
0x7fffffffe378  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe388  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe398  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3a8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3b8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3c8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3d8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3e8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3f8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe408  c290 0000 0000 0000 08e5 ffff ff7f 0000  ................
0x7fffffffe418  0000 0000 0200 0000 a005 4000 0000 0000  ..........@.....
0x7fffffffe428  2a30 a4f

In [340]:
# %r dr

In [ ]:
%r ds; pdf;

In [342]:
%r dc

dc
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
child stopped with signal 11
[+] SIGNAL 11 errno=0 addr=0x000090c2 code=1 ret=0
[0x000090c2]> 


# R2 Interact:

In [323]:
my_str = "aaaaaaaaaaaaaa"
%ri {my_str}

An r2 developer is coming to your place to manually analyze this program. Please wait for it

--press any key--



In [324]:
%r pdf

pdf
Usage: d   # Debug commands00l
| db[?]                    Breakpoints commands
| dbt[?]                   Display backtrace based on dbg.btdepth and dbg.btalgo
| dc[?]                    Continue execution
| dd[?]                    File descriptors (!fd in r1)
| de[-sc] [perm] [rm] [e]  Debug with ESIL (see de?)
| dg <file>                Generate a core-file (WIP)
| dH [handler]             Transplant process to a new handler
| di[?]                    Show debugger backend information (See dh)
| dk[?]                    List, send, get, set, signal handlers of child
| dL[?]                    List or set debugger handler
| dm[?]                    Show memory maps
| do[?]                    Open process (reload, alias for 'oo')
| doo[args]                Reopen in debugger mode with args (alias for 'ood')
| dp[?]                    List, attach to process or thread id
| dr[?]                    Cpu registers
| ds[?]                    Step, over, source line
| dt[?]             

# Find Target:

In [120]:
%r pdf @sym.normal

pdf @sym.normal
Invalid address (sym.normal)sym.normal
|ERROR| Invalid command 'pdf @sym.normal' (0x70)
[0x00400595]> 


In [121]:
%r pdf @sym.special

pdf @sym.special
Invalid address (sym.special)ym.special
|ERROR| Invalid command 'pdf @sym.special' (0x70)
[0x00400595]> 


# Other Notes: